In [1]:
# load
import pandas as pd
import lightgbm

data = pd.read_csv("X_train.csv", index_col=0)
data["mark"] = pd.read_csv("y_train.csv", index_col=0)["mark"]

stud_info = pd.read_csv("studs_info.csv", index_col=False)

X_validation = pd.read_csv("X_test.csv", index_col=0)

In [2]:
# rename columns
field_map = {
    "STD_ID": "stud",
    "НАПРАВЛЕНИЕ": "profession",
    "ГОД": "year",
    "АТТЕСТАЦИЯ": "exam_type",
    "ДИСЦИПЛИНА": "discipline",
    "КУРС": "course",
    "СЕМЕСТР": "semester",

    "   number": "number",
    "Пол": "sex",
    "Статус": "state",
    "Дата выпуска": "release_date",
    "Категория обучения": "category",
    "Форма обучения": "study_kind",
    "Шифр": "cipher",
    "направление (специальность)": "speciality",
    "   ": "what?",
    "Образование": "lvl_of_education",
    "Дата выдачи": "issue_date",
    "Что именно закончил": "education",
}

data.rename(columns=field_map, inplace=True)
X_validation.rename(columns=field_map, inplace=True)
stud_info.rename(columns=field_map, inplace=True)

In [3]:
stud_info.drop(stud_info[stud_info["stud"] == 92222].index, inplace=True)

# stud_info[np.isin(stud_info["number"], range(850, 900))].sort_values(by=["stud"])
# all(stud_info.groupby("speciality")["cipher"].nunique().eq(1))# and
all(stud_info.groupby("cipher")["speciality"].nunique().eq(1))
g = stud_info.groupby("speciality")["cipher"].nunique()
print(g[g != 1])

set(stud_info[stud_info["speciality"] == "Журналистика"]["cipher"])
# 203283

speciality
Журналистика                                                                 2
Зоотехния                                                                    2
Конструирование и технология электронных средств                             2
Конструкторско-технологическое обеспечение машиностроительных производств    2
Менеджмент                                                                   2
Прикладная математика и информатика                                          2
Радиотехника                                                                 2
Строительство                                                                2
Филология                                                                    2
Электроника и наноэлектроника                                                2
Юриспруденция                                                                2
Name: cipher, dtype: int64


{'42.03.02', '42.04.02'}

In [4]:
# remove duplicate entries (older ones)
stud_info = stud_info.sort_values(by=["stud", "issue_date"], na_position="first")
stud_info.drop_duplicates(subset=["stud"], keep="last", inplace=True)

import numpy as np
assert len(stud_info[np.isin(stud_info["stud"], stud_info[stud_info.duplicated(subset=["stud"])])]) == 0

In [5]:
%%capture
# clean up

# for each stud: year == course + const
# for each stud: course == ceil(semester / 2)
# therefore they are noise
fields = ["year", "course"]
data.drop(fields, axis=1, inplace=True)
X_validation.drop(fields, axis=1, inplace=True)

# all nulls and not present in data / validation
stud_info.drop(stud_info[stud_info["stud"] == 92222].index, inplace=True)

# for each stud: all number_s are equal
assert all(stud_info.groupby("number")["stud"].nunique().le(1)) and all(stud_info.groupby("stud")["number"].nunique().le(1))
fields = ["number", "issue_date", "release_date"]
stud_info.drop(fields, axis=1, inplace=True)

In [6]:
{
# ('НС', 'СР'): 4,
#  ('ОСН', 'СР'): 3,
#  ('НС', 'СП'): 5,
#  ('СР', 'СП'): 111,
#  ('ОСН', 'СП'): 24,
#  ('ОО', 'СР'): 22,
#  ('ОО', 'СП'): 131,
 ('НП', 'СР'): 1,
 ('НП', 'СП'): 10,
 ('СП', 'СП'): 7,
 ('СР', 'СР', 'СП'): 1,
 ('СР', 'СР'): 1,
 ('СП', 'СР'): 1,
 ('СП', 'НП'): 1}

# ('ОО', 'СР'      )
# (      'СР', 'СП')
# ('ОО',       'СП')

# ('ОО',        'СР', 'СП')
# (      'ОСН', 'СР'      )
# (      'ОСН', 'СП'      )

('ОО', 'ОСН', 'СР', 'СП')
('НС',        'СР'      )
('НС',              'СП')

# # SeriesGroupBy.cummax()

('НС', 'СП')

In [7]:
stud_info

,stud,sex,state,category,study_kind,cipher,speciality,what?,lvl_of_education,education
385,57258,М,СТ,ДОГ,З,15.03.05,Конструкторско-технологическое обеспечение маш...,10.0,СР,"ГИМНАЗИЯ N 2, НОВГОРОД"
4158,58419,М,СТ,ДОГ,З,11.03.03,Конструирование и технология электронных средств,155.0,СР,"СРЕДНЯЯ ШКОЛА N 8, НОВГОРОД"
4635,62101,М,СТ,ДОГ,З,13.03.01,Теплоэнергетика и теплотехника,144.0,СР,Пролетарская СОШ НОВГОРОДСКИЙ р-н
3248,65970,М,СТ,ДОГ,З,44.03.03,Специальное (дефектологическое) образование,200.0,СР,"СРЕДНЯЯ ШКОЛА N 2, НОВГОРОД"
5078,66520,М,СТ,ДОГ,З,40.03.01,Юриспруденция,197.0,СР,"СРЕДНЯЯ ШКОЛА N 8, НОВГОРОД"
...,...,...,...,...,...,...,...,...,...,...
2381,203270,М,СТ,ДОГ,З,13.03.01,Теплоэнергетика и теплотехника,8879.0,НП,пту 15
2380,203271,М,СТ,ДОГ,З,13.03.01,Теплоэнергетика и теплотехника,8307.0,ОО,"СРЕДНЯЯ ШКОЛА N 9, НОВГОРОД"
1638,203272,М,СТ,ДОГ,З,13.03.01,Теплоэнергетика и теплотехника,NaN,NaN,NaN
4823,203283,М,СТ,ДОГ,Д,07.03.01,Архитектура,NaN,NaN,NaN


In [8]:
%%capture
stud_info.fillna({"lvl_of_education": "НЕТ", "what?": 0.0}, inplace=True)

data = data.merge(stud_info, how="left", on="stud")
X_validation = X_validation.merge(stud_info, how="left", on="stud")

In [9]:
data

,stud,profession,semester,exam_type,discipline,mark,sex,state,category,study_kind,cipher,speciality,what?,lvl_of_education,education
0,175711,38.03.02 Менеджмент,4.0,Экзамен,Маркетинг,5,М,СТ,БП,Д,38.03.02,Менеджмент,5445.0,СР,"СРЕДНЯЯ ШКОЛА N 31, НОВГОРОД"
1,100647,31.05.01 Лечебное дело,5.0,Экзамен,"Пропедевтика внутренних болезней, лучевая диаг...",5,Ж,ВЫП,ДОГ,Д,31.05.01,Лечебное дело,2668.0,СР,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА
2,199311,31.05.03 Стоматология,1.0,Экзамен,Химия,5,Ж,СТ,БП,Д,31.05.03,Стоматология,11431.0,СР,"ГИМНАЗИЯ N 2, НОВГОРОД"
3,162002,38.03.02 Менеджмент,3.0,Экзамен,Статистика,4,М,СТ,БП,Д,38.03.02,Менеджмент,4981.0,СР,"Гимназия ""ГАРМОНИЯ"""
4,108384,31.05.01 Лечебное дело,9.0,Экзамен,Педиатрия,3,Ж,СТ,ДОГ,Д,31.05.01,Лечебное дело,2034.0,СР,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105053,176141,40.03.01 Юриспруденция,2.0,Курсовая работа,Конституционное право,3,Ж,СТ,ДОГ,В,40.03.01,Юриспруденция,8613.0,СР,"МОУ СОШ N 1, СОЛЬЦЫ с филиалом в Куклино"
105054,182794,31.05.01 Лечебное дело,4.0,Экзамен,Нормальная физиология,3,Ж,СТ,ДОГ,Д,31.05.01,Лечебное дело,5505.0,СР,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА
105055,162391,31.05.01 Лечебное дело,2.0,Дифференцированный зачет,Иностранный язык,4,Ж,СТ,БП,Д,31.05.01,Лечебное дело,4900.0,СР,"ГИМНАЗИЯ N 1, НОВГОРОД"
105056,202479,38.03.01 Экономика,1.0,Дифференцированный зачет,Информатика,4,Ж,СТ,ДОГ,З,38.03.01,Экономика,5466.0,СР,"Гимназия ""ГАРМОНИЯ"""


In [10]:

# encode labels
from sklearn import preprocessing

fields = ["discipline", "profession", "exam_type", "sex", "category", "speciality", "education", "state", "cipher"]
le_s = {
    field_name: preprocessing.LabelEncoder().fit(pd.concat([data[field_name], X_validation[field_name]]))
    for field_name in fields}

order = [
    "НЕТ",  # 190  Нет данных
    "ОО",   # 160  Начальное общее образование
    "ОСН",  # 32   Основное общее образование
    "НС",   # 14   Неполное среднее образование
    "СР",   # 4101 Среднее общее образование
    "НВ",   # 2    Неполное высшее образование
    "НП",   # 50   Начальное/Незаконченное? профессиональное образование
    "СП",   # 916  Среднее профессиональное образование
]
le_s["lvl_of_education"] = preprocessing.LabelEncoder().fit(order)

order = ["В", "Д", "З"]  # вечернее, дневное, заочное
le_s["study_kind"] = preprocessing.LabelEncoder().fit(order)

for field_name, le in le_s.items():
    data[field_name] = le.transform(data[field_name])
    X_validation[field_name] = le.transform(X_validation[field_name])

# 69.0 to 69
fields = ["semester", "what?"]
for field_name in fields:
    data[field_name] = data[field_name].astype(int)
    X_validation[field_name] = X_validation[field_name].astype(int)

# normalize
data["semester"] -= 1
X_validation["semester"] -= 1

In [11]:
data

,stud,profession,semester,exam_type,discipline,mark,sex,state,category,study_kind,cipher,speciality,what?,lvl_of_education,education
0,175711,36,3,4,408,5,1,1,0,1,36,19,5445,7,185
1,100647,24,4,4,946,5,0,0,1,1,24,17,2668,7,39
2,199311,25,0,4,1326,5,0,1,0,1,25,29,11431,7,23
3,162002,36,2,4,1100,4,1,1,0,1,36,19,4981,7,32
4,108384,24,8,4,713,3,0,1,1,1,24,17,2034,7,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105053,176141,40,1,2,364,3,0,1,1,0,40,43,8613,7,68
105054,182794,24,3,4,556,3,0,1,1,1,24,17,5505,7,39
105055,162391,24,1,0,228,4,0,1,0,1,24,17,4900,7,22
105056,202479,35,0,0,238,4,0,1,1,2,35,40,5466,7,32


In [12]:
fields = ["stud", "profession", "discipline", "speciality", "education", "cipher"]
for field_name in fields:
    mean_mark = data.groupby(field_name).mean()["mark"]

    mean_name = field_name + "_mean"
    data[mean_name] = data[field_name].map(mean_mark)
    X_validation[mean_name] = X_validation[field_name].map(mean_mark)

# create dummy variables

columns = []

data = pd.get_dummies(data, columns=columns)
#del data ['study_kind']
#del data ['exam_type']

X_validation = pd.get_dummies(X_validation, columns=columns)

# remove unneeded data
# use previous fields
# fields = ["stud", "discipline", "profession"]
data.drop(fields, axis=1, inplace=True)
X_validation.drop(fields, axis=1, inplace=True)

In [13]:
data

,semester,exam_type,mark,sex,state,category,study_kind,what?,lvl_of_education,stud_mean,profession_mean,discipline_mean,speciality_mean,education_mean,cipher_mean
0,3,4,5,1,1,0,1,5445,7,4.181818,4.157895,4.326389,4.170145,4.228495,4.157895
1,4,4,5,0,0,1,1,2668,7,4.116279,4.205835,4.266444,4.205835,4.125923,4.205835
2,0,4,5,0,1,0,1,11431,7,4.750000,4.119935,3.569940,4.119935,4.335843,4.119935
3,2,4,4,1,1,0,1,4981,7,4.172414,4.157895,3.721519,4.170145,4.329181,4.157895
4,8,4,3,0,1,1,1,2034,7,3.542857,4.205835,4.333919,4.205835,4.125923,4.205835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105053,1,2,3,0,1,1,0,8613,7,3.615385,3.951955,3.833866,3.996726,4.063981,3.951955
105054,3,4,3,0,1,1,1,5505,7,4.052632,4.205835,3.693413,4.205835,4.125923,4.205835
105055,1,0,4,0,1,0,1,4900,7,4.458333,4.205835,4.153555,4.205835,4.278993,4.205835
105056,0,0,4,0,1,1,2,5466,7,4.187500,4.114599,3.984946,4.114599,4.329181,4.114599


In [14]:

marks = data.pop("mark")


# создание и обучение модели на основе бустинга рещающих деревьев
import tpot
from sklearn.model_selection import StratifiedKFold
from tpot import TPOTClassifier

cv = StratifiedKFold(n_splits=5)
# define TPOTClassifier
model = TPOTClassifier(generations=5, population_size=20, cv=cv, scoring='accuracy', verbosity=2, random_state=1, n_jobs=-1)
# performing the search for best fit
model.fit(data, marks)

Optimization Progress:   0%|          | 0/120 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.6777589477642828

Generation 2 - Current best internal CV score: 0.6777589477642828

Generation 3 - Current best internal CV score: 0.6861257761485664

Generation 4 - Current best internal CV score: 0.6881627409492963

Generation 5 - Current best internal CV score: 0.6895524402504238

Best pipeline: RandomForestClassifier(input_matrix, bootstrap=True, criterion=gini, max_features=0.4, min_samples_leaf=5, min_samples_split=15, n_estimators=100)


TPOTClassifier(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
               generations=5, n_jobs=-1, population_size=20, random_state=1,
               scoring='accuracy', verbosity=2)

In [18]:
from sklearn.metrics import mean_absolute_error, r2_score


y_pred = pd.read_csv("sample_submission.csv", index_col=0)
y_pred["mark"] = model.predict(X_validation)

y_pred.to_csv("baseline_submission.csv")


Imputing missing values in feature set
